In [68]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import ndcg_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [69]:
data = pd.read_csv('filled_training_set.csv')

In [70]:
query_lengths = data.groupby('srch_id').size().tolist()
print(query_lengths[:10])

[28, 32, 5, 21, 33, 28, 31, 29, 33, 34]


In [71]:
# 生成查询信息
query_info = query_lengths

In [72]:
target_name = 'score'
# feature=['prop_country_id','prop_id','prop_starrating','prop_review_score','prop_location_score1','prop_location_score2','prop_review_score','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool']
# X=data[feature]
X = data.drop(columns=[target_name])
y = data[target_name]

In [73]:
x_train_np = X.to_numpy()
y_train_np = y.to_numpy()
# X_train_base, X_meta, y_train_base, y_meta = train_test_split(x_train_np, y_train_np, test_size=0.1, random_state=42)


In [74]:
train_data = lgb.Dataset(x_train_np, label=y_train_np,group=query_info)

In [45]:
# # 2. 选择基础模型
# base_models = [
#     lgb.LGBMRanker(objective='lambdarank', metric='ndcg', num_leaves=31, learning_rate=0.05, verbose=0),
#     # RandomForestRegressor(n_estimators=100, random_state=42),  # 随机森林回归器
#     # GradientBoostingRegressor(n_estimators=100, random_state=42)  # 梯度提升回归器
# ]

In [46]:
# # 3. 训练基础模型
# meta_features = []
# for model in base_models:
#     if isinstance(model, lgb.LGBMRanker):
#         model.fit(x_train_np, y_train_np, group=query_info)
#     else:
#         model.fit(x_train_np, y_train_np)
#     meta_features.append(model.predict(X_meta))

In [47]:
# # 4. 生成次级训练集
# X_meta_stacking = np.column_stack(meta_features)

# # 5. 选择次级模型
# meta_model = LinearRegression()

# # 6. 训练次级模型
# meta_model.fit(X_meta_stacking, y_meta)

LinearRegression()

In [75]:
def merge_comp(df):
    comp_rates = ['comp'+str(i)+'_rate' for i in range(1, 9)]
    comp_invs = ['comp'+str(i)+'_inv' for i in range(1, 9)]
    comp_rate_percent_diffs = ['comp'+str(i)+'_rate_percent_diff' for i in range(1, 9)]

    df['comp_avg_rate'] = df[comp_rates].mean(axis=1)
    df['comp_avg_inv'] = df[comp_invs].mean(axis=1)
    df['comp_avg_rate_percent_diff'] = df[comp_rate_percent_diffs].mean(axis=1)

    comps = comp_rates + comp_invs + comp_rate_percent_diffs
    df.drop(comps, axis=1, inplace=True)

In [76]:
def parse_date(df):
    df.date_time = pd.to_datetime(df.date_time)
    df['year'] = df.date_time.dt.year
    df['month'] = df.date_time.dt.month

In [78]:
test_data = pd.read_csv('test_set_VU_DM.csv')
parse_date(test_data)
data_without_date = test_data.drop(columns=['date_time', 'orig_destination_distance'])
merge_comp(data_without_date)
# iterative_imputer = IterativeImputer(estimator=BayesianRidge(),max_iter=2, random_state=0)
# filled_data = iterative_imputer.fit_transform(data_without_date)
imputer = SimpleImputer(strategy='constant', fill_value=-10)
print(imputer)
filled_data = imputer.fit_transform(data_without_date)
filled_data = pd.DataFrame(filled_data, columns=data_without_date.columns)
print(data_without_date.columns)
# filled_data=filled_data[feature]
x_test_np = filled_data.to_numpy()

SimpleImputer(fill_value=-10, strategy='constant')
Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score', 'random_bool',
       'year', 'month', 'comp_avg_rate', 'comp_avg_inv',
       'comp_avg_rate_percent_diff'],
      dtype='object')


In [66]:
# # 7. 模型融合
# def stacking_predict(models, meta_model, X):
#     meta_features = []
#     for model in models:
#         meta_features.append(model.predict(X))
#     X_meta = np.column_stack(meta_features)
#     return meta_model.predict(X_meta)

# # 8. 评估性能
# y_pred = stacking_predict(base_models, meta_model, x_test_np)

# ndcg = ndcg_score(y_test, y_pred)

# # 输出NDCG得分
# print("NDCG score on test set:", ndcg)

In [67]:
result = test_data[['srch_id', 'prop_id']]
result['score'] = y_pred
# result['label'] = label
result['rank'] = result.groupby('srch_id')['score'].rank(ascending=False) 
sorted_df = result.sort_values(by=['srch_id', 'score'], ascending=[True, False])

# 输出sid和pid
output = sorted_df[['srch_id', 'prop_id']]
output.to_csv('6.csv', index=False)

/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/ipykernel_957/4061822482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['score'] = y_pred
/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/ipykernel_957/4061822482.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rank'] = result.groupby('srch_id')['score'].rank(ascending=False)


In [77]:
# params = {
#     'objective': 'lambdarank',
#     'metric': 'ndcg',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'verbose': 0
# }
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'num_leaves': 64,  # 增加树的深度
    'learning_rate': 0.1,  # 增加学习率
    'max_depth': -1,  # 不限制树的深度
    'min_data_in_leaf': 50,  # 设置叶子节点的最小数据量
    'lambda_l1': 0.1,  # 增加L1正则化项
    'lambda_l2': 0.1,  # 增加L2正则化项
    'feature_fraction': 0.8,  # 设置特征抽样比例
    'bagging_fraction': 0.8,  # 设置数据抽样比例
    'bagging_freq': 5,  # 每5次迭代进行一次数据抽样
    'verbose': 0
}

gbm = lgb.train(params, train_data , num_boost_round=100)

# Save the trained model if needed
gbm.save_model('ranking_model2.txt')

In [79]:
# 加载模型
model = lgb.Booster(model_file='ranking_model2.txt')

# 使用加载的模型进行预测
prediction= model.predict(x_test_np)

In [80]:
# label = test_data.score

result = test_data[['srch_id', 'prop_id']]
result['score'] = prediction
# result['label'] = label
result['rank'] = result.groupby('srch_id')['score'].rank(ascending=False) 
sorted_df = result.sort_values(by=['srch_id', 'score'], ascending=[True, False])

# 输出sid和pid
output = sorted_df[['srch_id', 'prop_id']]
output.to_csv('4.csv', index=False)

/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/ipykernel_957/3737440317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['score'] = prediction
/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/ipykernel_957/3737440317.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rank'] = result.groupby('srch_id')['score'].rank(ascending=False)


In [4]:
pip install lightgbm

  Using cached lightgbm-4.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for lightgbm (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      2024-05-10 22:19:52,007 - scikit_build_core - INFO - RUN: /private/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/pip-build-env-kgic8tmm/normal/lib/python3.11/site-packages/cmake/data/bin/cmake -E capabilities
      2024-05-10 22:19:52,013 - scikit_build_core - INFO - CMake version: 3.29.2
      *** scikit-build-core 0.9.3 using CMake 3.29.2 (wheel)
      2024-05-10 22:19:52,015 - scikit_build_core - INFO - Build directory: /private/var/folders/g6/tqykz2q510j48yls1qryzrkw0000gn/T/tmpci6cthiv/build
      *** Configuring CMake...
      2024-05-10 22:19:52,023 - scikit_build_core - INFO - RUN: /private/va

In [5]:
pip uninstall lightgbm

Note: you may need to restart the kernel to use updated packages.


In [6]:
conda install -c conda-forge lightgbm

Retrieving notices: ...working... done
Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::jupyterlab_server==2.22.0=py311hca03da5_0
  - defaults/osx-arm64::_anaconda_depends==2023.09=py311_openblas_1
  - defaults/noarch::argon2-cffi==21.3.0=pyhd3eb1b0_0
  - defaults/osx-arm64::notebook==6.5.4=py311hca03da5_1
  - defaults/osx-arm64::twisted==22.10.0=py311h80987f9_0
  - defaults/osx-arm64::jupyter_server_ydoc==0.8.0=py311hca03da5_1
  - defaults/osx-arm64::jupyterlab==3.6.3=py311hca03da5_0
  - defaults/osx-arm64::s3fs==2023.4.0=py311hca03da5_0
  - defaults/osx-arm64::datasets==2.12.0=py311hca03da5_0
  - defaults/osx-arm64::huggingface_hub==0.15.1=py311hca03da5_0
  - defaults/osx-arm64::anaconda-navigator==2.5.0=py311hca03da5_0
  - defaults/osx-arm64::notebook-shim==0.2.2=py311hca03da5_0
  - defaults/osx-arm64::transformers==4.32.1=py311hca03da5_0
  - defaults/osx-ar